In [129]:
import pandas as pd
import datetime as datetime

In [130]:
securityData = pd.read_csv('iseeData.csv')
index = pd.read_csv('historicalSANDP.csv')

In [131]:
securityData['DATE'] = pd.to_datetime(securityData['DATE'])
index['Date'] = pd.to_datetime(index['Date'])

In [132]:
securityData.head()

,DATE,ALL SECURITIES,10 AVG,20 AVG,50 AVG,ALL EQUITIES,10 AVG.1,20 AVG.1,50 AVG.1,ALL ETFS,10 AVG.2,20 AVG.2,50 AVG.2
0,2006-01-03,149,0,0,0,212,0,0,0,49,0,0,0
1,2006-01-04,180,0,0,0,218,0,0,0,74,0,0,0
2,2006-01-05,195,0,0,0,264,0,0,0,38,0,0,0
3,2006-01-06,144,0,0,0,226,0,0,0,28,0,0,0
4,2006-01-09,188,0,0,0,256,0,0,0,45,0,0,0


In [160]:
index.head()

,Date,Open,High,Low,Close,Adj Close,Volume,priceShift,target,ISE All Securities,ISE All Equites
0,2006-01-03,1248.290039,1270.219971,1245.739990,1268.800049,1268.800049,2554570000,-1268.800049,0,149,212
1,2006-01-04,1268.800049,1275.369995,1267.739990,1273.459961,1273.459961,2515330000,-4.659912,0,180,218
2,2006-01-05,1273.459961,1276.910034,1270.300049,1273.479980,1273.479980,2433340000,-0.020019,0,195,264
3,2006-01-06,1273.479980,1286.089966,1273.479980,1285.449951,1285.449951,2446560000,-11.969971,0,144,226
4,2006-01-09,1285.449951,1290.780029,1284.819946,1290.150024,1290.150024,2301490000,-4.700073,0,188,256


In [134]:
#adding target variable to s&p data 0 if price goes down 1 if price goes up 
increment = [0] + index['Close'].values.tolist()
index['priceShift'] = increment[:-1] - index['Close'] 
index['target'] = index['priceShift'].apply(lambda x: 0 if x < 0 else 1)

In [135]:
#merging dataframes
index['ISE All Securities'] = securityData['ALL SECURITIES']
index['ISE All Equites'] = securityData['ALL EQUITIES']

In [136]:
from sklearn.linear_model import LogisticRegression

In [138]:
model = LogisticRegression()

In [139]:
train = index.iloc[:2000].reset_index(drop=True)

In [140]:
test = index.iloc[2000:].reset_index(drop=True)

In [151]:
### baseline
test['target'].value_counts()[0]/len(test)

0.5342019543973942

In [142]:
features = ['ISE All Securities', 'ISE All Equites']

In [143]:
model.fit(train[features], train['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [144]:

predictions = model.predict(test[features])

In [145]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [146]:
confusion_matrix(test['target'], predictions)
accuracy_score(test['target'], predictions)

0.5684039087947883

In [152]:
plt.clf()
plot_confusion_matrix(confusion_matrix(test['target'], predictions), classes=['price up', 'price down'])


plt.savefig('results.jpg')

Confusion matrix, without normalization
[[251 405]
 [125 447]]


In [21]:
from sklearn.metrics import roc_auc_score



In [22]:
roc_auc_score(test['target'], predictions)

0.5820452413440218

In [23]:
matthews_corrcoef(test['target'], predictions)

0.17759044961832265

### predicting from reddit data

In [215]:
import json
with open('data/titleData.json') as x:
    unstructured = json.loads(x.read())

In [216]:
len(unstructured)

4523

In [217]:
dates = list(unstructured.keys())
datetime = pd.to_datetime(list(unstructured.keys()))

unstructured['2016-10-09']

[{'author': 'cdpaper',
  'author_flair_css_class': None,
  'author_flair_text': None,
  'body': '&gt; Is there an algorithm used to read reports and make decisions\n\nOh honey...',
  'created_utc': 1475886820,
  'id': 'd8in94a',
  'link_id': 't3_56anrb',
  'parent_id': 't1_d8i2vh0',
  'score': 4,
  'stickied': False,
  'subreddit': 'finance',
  'subreddit_id': 't5_2qhfj'},
 {'author': 'cdpaper',
  'author_flair_css_class': None,
  'author_flair_text': None,
  'body': 'Do you trade with a mouse or do you use bots?',
  'created_utc': 1475886915,
  'id': 'd8inb5e',
  'link_id': 't3_56anrb',
  'parent_id': 't1_d8hyn9v',
  'score': 2,
  'stickied': False,
  'subreddit': 'finance',
  'subreddit_id': 't5_2qhfj'},
 {'author': 'tagus',
  'author_flair_css_class': None,
  'author_flair_text': None,
  'body': '&gt; Once you feel comfortable with that, dip a toe into the real fx market \n\nHow much are we talking, here?  Also, what books?',
  'created_utc': 1475888173,
  'id': 'd8io1mw',
  'link_i

In [218]:
newDict = {}
for x,y in zip(dates, datetime):
    newDict[y] = ' '.join([comment['title'] for comment in unstructured[x]])

KeyError: 'title'

In [ ]:
index['words'] = index['Date'].apply(lambda x: newDict[x] if x in newDict.keys() else 'null')
index['words'] = index['words'].apply(lambda x: 'null' if x == '' else x)

In [ ]:
withWords = index[index['words'] != 'null'].reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', AdaBoostClassifier(learning_rate = 1, n_estimators = 300, algorithm = 'SAMME'))])



In [ ]:
train = withWords.iloc[0:2000].reset_index(drop=True)

test = withWords.iloc[2000:].reset_index(drop=True)

In [ ]:
pipeline.fit(train['words'], train['target'])
results = pipeline.predict(test['words'])

In [ ]:
test['target'].value_counts()

258/len(test)

In [ ]:
plot_confusion_matrix(confusion_matrix(test['target'], results), classes=['price up', 'price down'])

In [ ]:
roc_auc_score(test['target'], results)

In [ ]:
matthews_corrcoef(test['target'], results)

In [ ]:
accuracy_score(test['target'], results)

In [ ]:
pd.Series(test['target'] == results).value_counts()

In [ ]:
redditTitles